In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

In [4]:
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

In [5]:
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [6]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [7]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

In [8]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


## Kevin,

Okay. So I have spent an obscene amount of time on trying to create bigrams and trigrams of these corpuses. I probably should have moved on a week ago. I even took a pause in my learning. I have done a lot of work on collocations in the last couple weeks and there is just something about these corpuses that do... not... work with it. So I've decided to move on. 

You and I worked on creaeting features that accound for the POS of the sentences. But we never really got it to work because of the randomization aspect. 

I'm going to process the cells below to improve the models using the sentence length as an additional feature of the model. Yes, it would be GREAT if we could somehow capture the vernacular of an author based on their predilection for certain sentence structures. Maybe we pick that up later down the road. I can definitely see the value. But for now I'm going to increase the number of common words from 2000 to 2500 and add sentence length. I'll remove all other aspects of this notebook for the sanity of anyone reading it. 

In [9]:
# Utility function to create a list of the 2500 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2500)]

In [10]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [11]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

In [12]:
# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [13]:
word_counts = bow_features(sentences, common_words)
word_counts['sents_len'] = word_counts['text_sentence'].apply(lambda x: len(x))

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


In [14]:
word_counts.head(3)

,firmness,comparatively,unfounded,friendliness,warning,alicia,evening,dependent,guest,relation,...,cold,intend,propriety,rally,not,root,continue,text_sentence,text_source,sents_len
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33


In [20]:
y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Random Forest

In [22]:
rfc = ensemble.RandomForestClassifier()

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9889515749882464

Test set score: 0.900375939849624


## Logistic Regression

In [23]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(4254, 3477) (4254,)
Training set score: 0.9565115185707569

Test set score: 0.9088345864661654


## Gradient Boosting

In [24]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8843441466854725

Test set score: 0.8740601503759399


## SVM

In [25]:
svm = SVC(kernel = 'linear', tol=0.001)
print(svm.fit(X_train, y_train))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [32]:
# svm.score(X_test, y_test)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.9727315467795017

Test set score: 0.9088345864661654


### Cross-validations

In [27]:
cross_val_score(rfc, X_train, y_train, cv=5)

array([0.89424207, 0.8907168 , 0.88249119, 0.8907168 , 0.88352941])

In [28]:
cross_val_score(lr, X_train, y_train, cv=5)

array([0.92009401, 0.92009401, 0.90364277, 0.90246769, 0.90823529])

In [29]:
cross_val_score(clf, X_train, y_train, cv=5)

array([0.88366627, 0.87191539, 0.8613396 , 0.86603995, 0.86352941])

In [30]:
cross_val_score(svm, X_train, y_train, cv=5)

array([0.90364277, 0.92009401, 0.90246769, 0.89894242, 0.90588235])